## Imports

In [24]:
import pandas as pd
import numpy as np
import unicodedata
import re

In [22]:
df = pd.read_csv('./Data/RepublicaGuatemala.csv')

In [153]:
df.head()

,CODIGO,DISTRITO,DEPARTAMENTO,MUNICIPIO,ESTABLECIMIENTO,DIRECCION,TELEFONO,SUPERVISOR,DIRECTOR,NIVEL,SECTOR,AREA,STATUS,MODALIDAD,JORNADA,PLAN,DEPARTAMENTAL
0,16-01-0138-46,16-031,ALTA VERAPAZ,COBAN,COLEGIO COBAN,KM.2 SALIDA A SAN JUAN CHAMELCO ZONA 8,77945104,PATRICIO NAJARRO ASENCIO,GUSTAVO ADOLFO SIERRA POP,DIVERSIFICADO,PRIVADO,URBANA,ABIERTA,MONOLINGUE,MATUTINA,DIARIO(REGULAR),ALTA VERAPAZ
1,16-01-0139-46,16-031,ALTA VERAPAZ,COBAN,COLEGIO PARTICULAR MIXTO VERAPAZ,KM 209.5 ENTRADA A LA CIUDAD,77367402,PATRICIO NAJARRO ASENCIO,GILMA DOLORES GUAY PAZ DE LEAL,DIVERSIFICADO,PRIVADO,URBANA,ABIERTA,MONOLINGUE,MATUTINA,DIARIO(REGULAR),ALTA VERAPAZ
2,16-01-0140-46,16-031,ALTA VERAPAZ,COBAN,"COLEGIO ""LA INMACULADA""",7A. AVENIDA 11-109 ZONA 6,78232301,PATRICIO NAJARRO ASENCIO,VIRGINIA SOLANO SERRANO,DIVERSIFICADO,PRIVADO,URBANA,ABIERTA,MONOLINGUE,MATUTINA,DIARIO(REGULAR),ALTA VERAPAZ
3,16-01-0141-46,16-005,ALTA VERAPAZ,COBAN,ESCUELA NACIONAL DE CIENCIAS COMERCIALES,2A CALLE 11-10 ZONA 2,79514215,NORA LILIANA FIGUEROA HERNÁNDEZ,HÉCTOR ROLANDO CHUN POOU,DIVERSIFICADO,OFICIAL,URBANA,ABIERTA,MONOLINGUE,MATUTINA,DIARIO(REGULAR),ALTA VERAPAZ
4,16-01-0142-46,16-005,ALTA VERAPAZ,COBAN,INSTITUTO NORMAL MIXTO DEL NORTE 'EMILIO ROSAL...,3A AVE 6-23 ZONA 11,79521468,NORA LILIANA FIGUEROA HERNÁNDEZ,VICTOR HUGO DOMÍNGUEZ REYES,DIVERSIFICADO,OFICIAL,URBANA,ABIERTA,BILINGUE,VESPERTINA,DIARIO(REGULAR),ALTA VERAPAZ


# CODIGO

In [154]:
df['DISTRITO'] = df['DISTRITO'].astype(str).str.upper().str.strip()

df['DISTRITO'] = df['DISTRITO'].str.replace(r'[^0-9\-]', '', regex=True)

pattern = re.compile(r'^\d{2}-\d{3}$')
mask_valid = df['DISTRITO'].str.match(pattern)
df.loc[~mask_valid, 'DISTRITO'] = np.nan

n_invalid = df['DISTRITO'].isna().sum()
print(f"{n_invalid} registros de DISTRITO quedaron como NaN tras la validación.")

7 registros de DISTRITO quedaron como NaN tras la validación.


# DISTRITO

In [155]:
df['DISTRITO'] = df['DISTRITO'].astype(str).str.upper().str.strip()

df['DISTRITO'] = df['DISTRITO'].str.replace(r'[^0-9\-]', '', regex=True)

pattern = re.compile(r'^\d{2}-\d{3}$')
mask_valid = df['DISTRITO'].str.match(pattern)
df.loc[~mask_valid, 'DISTRITO'] = np.nan

n_invalid = df['DISTRITO'].isna().sum()
print(f"{n_invalid} registros de DISTRITO quedaron como NaN tras la validación.")

print(df['DISTRITO'].head(20))

7 registros de DISTRITO quedaron como NaN tras la validación.
0     16-031
1     16-031
2     16-031
3     16-005
4     16-005
5     16-031
6     16-031
7     16-005
8     16-031
9     16-031
10    16-031
11    16-005
12    16-005
13    16-031
14    16-031
15    16-031
16    16-031
17    16-031
18    16-005
19    16-031
Name: DISTRITO, dtype: object


# DEPARTAMENTO

In [156]:
def remove_accents(text):
    nkfd_form = unicodedata.normalize('NFKD', text)
    return ''.join([c for c in nkfd_form if not unicodedata.combining(c)])

df['DEPARTAMENTO'] = (
    df['DEPARTAMENTO']
    .astype(str)
    .str.upper()
    .str.strip()
    .apply(remove_accents)
)

replacements = {
    'SACATEPEQUEZ': 'SACATEPEQUEZ',
}
df['DEPARTAMENTO'] = df['DEPARTAMENTO'].replace(replacements)

catalogo = (
    pd.read_csv('./Data/catalogo_departamentos.csv')['DEPARTAMENTO']
      .astype(str)
      .str.upper()
      .str.strip()
      .apply(remove_accents)
      .tolist()
)

mask_valid = df['DEPARTAMENTO'].isin(catalogo)
invalid_values = df.loc[~mask_valid, 'DEPARTAMENTO'].unique()
if len(invalid_values) > 0:
    print("Los siguientes DEPARTAMENTO no coinciden con el catálogo oficial:")
    for val in invalid_values:
        print(f" - {val}")

# 4) Reemplazar los string 'NAN' o '' por valores reales NaN
df['DEPARTAMENTO'].replace({'NAN': np.nan, '': np.nan}, inplace=True)

Los siguientes DEPARTAMENTO no coinciden con el catálogo oficial:
 - NAN


C:\Users\thiag\AppData\Local\Temp\ipykernel_9456\538770638.py:35: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['DEPARTAMENTO'].replace({'NAN': np.nan, '': np.nan}, inplace=True)


# MUNICIPIO

In [157]:
def remove_accents(text):
    """Elimina acentos y tildes de una cadena."""
    nkfd_form = unicodedata.normalize('NFKD', text)
    return ''.join(c for c in nkfd_form if not unicodedata.combining(c))

df['MUNICIPIO'] = (
    df['MUNICIPIO']
    .astype(str)
    .str.upper()
    .str.strip()
    .apply(remove_accents)
    .str.replace(r'[^A-Z\s\-]', '', regex=True)
)

df['MUNICIPIO'].replace({'': np.nan, 'NAN': np.nan}, inplace=True)

mun_counts = df['MUNICIPIO'].value_counts(dropna=False)
print("=== MUNICIPIOS ÚNICOS Y FRECUENCIAS ===")
print(mun_counts.to_string())

low_freq = mun_counts[mun_counts < 5]
print("\n=== MUNICIPIOS CON POCAS OCURRENCIAS (conteo < 5) ===")
print(low_freq.to_string())

=== MUNICIPIOS ÚNICOS Y FRECUENCIAS ===
MUNICIPIO
ZONA                            867
MIXCO                           300
VILLA NUEVA                     223
QUETZALTENANGO                  175
RETALHULEU                      156
CHIMALTENANGO                   138
MAZATENANGO                     126
MORALES                          98
ESCUINTLA                        97
PUERTO BARRIOS                   95
SAN MIGUEL PETAPA                94
JUTIAPA                          90
HUEHUETENANGO                    86
COBAN                            80
AMATITLAN                        78
SANTA LUCIA COTZUMALGUAPA        74
COATEPEQUE                       74
SAN JUAN SACATEPEQUEZ            71
ANTIGUA GUATEMALA                67
MALACATAN                        67
JALAPA                           63
VILLA CANALES                    53
LA LIBERTAD                      48
SAN JOSE PINULA                  46
SAN PEDRO SACATEPEQUEZ           45
SAN MARCOS                       44
CHIQUIMULA    

C:\Users\thiag\AppData\Local\Temp\ipykernel_9456\986716668.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['MUNICIPIO'].replace({'': np.nan, 'NAN': np.nan}, inplace=True)


# ESTABLECIMIENTO

In [158]:
df['ESTABLECIMIENTO'] = df['ESTABLECIMIENTO'].astype(str).str.strip().str.upper()

df['ESTABLECIMIENTO'] = df['ESTABLECIMIENTO'] \
    .str.replace(r"[\"\'\(\)]", "", regex=True) \
    .str.replace(r"[^A-Z0-9\s\-]", "", regex=True)

abbrev_map = {
    r"\bLICEO\b": "LICEO",
    r"\bCOLEGIO\b": "COLEGIO",
    r"\bINSTITUTO\b": "INSTITUTO",
    r"\bESCUELA\b": "ESCUELA",
}
for pat, rep in abbrev_map.items():
    df['ESTABLECIMIENTO'] = df['ESTABLECIMIENTO'].str.replace(pat, rep, regex=True)

df['ESTABLECIMIENTO'] = df['ESTABLECIMIENTO'] \
    .str.replace(r"-LIEC-", "LIEC", regex=True) \
    .str.replace(r"\-+", "-", regex=True)

df['ESTABLECIMIENTO'] = df['ESTABLECIMIENTO'] \
    .str.replace(r"\s+", " ", regex=True) \
    .str.strip()

df['ESTABLECIMIENTO'].replace({'': np.nan, 'NAN': np.nan}, inplace=True)

print(df['ESTABLECIMIENTO'].head(20))

0                                         COLEGIO COBAN
1                      COLEGIO PARTICULAR MIXTO VERAPAZ
2                                 COLEGIO LA INMACULADA
3              ESCUELA NACIONAL DE CIENCIAS COMERCIALES
4     INSTITUTO NORMAL MIXTO DEL NORTE EMILIO ROSALE...
5                     COLEGIO PARTICULAR MIXTO IMPERIAL
6                                  LICEO MODERNO LATINO
7         INSTITUTO NACIONAL DE EDUCACION DIVERSIFICADA
8                       COLEGIO DE INFORMATICA CENINFAV
9                             LICEO AMERICANO DEL NORTE
10                            LICEO AMERICANO DEL NORTE
11    INSTITUTO NORMAL PREPRIMARIA BILINGUE ADSCRITO...
12    INSTITUTO NORMAL PRIMARIA BILINGUE INTERCULTUR...
13         CENTRO DE FORMACION INTEGRAL LAS CONCHAS FMG
14           INSTITUTO MIXTO PRIVADO SALUD Y DESARROLLO
15              INSTITUTO PRIVADO RAUL CHENAL HEINEMANN
16                          LICEO MIGUEL ANGEL ASTURIAS
17                     COLEGIO CRISTIANO BILINGU

C:\Users\thiag\AppData\Local\Temp\ipykernel_9456\2256456406.py:24: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['ESTABLECIMIENTO'].replace({'': np.nan, 'NAN': np.nan}, inplace=True)


# DIRECCION

In [159]:
df['DIRECCION'] = (
    df['DIRECCION']
    .astype(str)
    .str.upper()
    .str.strip()
    .str.replace(r'\s+', ' ', regex=True)
)

df['DIRECCION'] = df['DIRECCION'].str.replace(r'[\"\'\(\)]', '', regex=True)

abbr_map = {
    r'\bC/': 'CALLE ',
    r'\bCL\b': 'CALLE',
    r'\bZ\b': 'ZONA',
    r'\bALDEA\b': 'ALDEA',
}
for pat, rep in abbr_map.items():
    df['DIRECCION'] = df['DIRECCION'].str.replace(pat, rep, regex=True)

df['DIRECCION'] = (
    df['DIRECCION']
    .str.replace(r'\bZN\b', 'ZONA', regex=True)
    .str.replace(r'\bALD\b', 'ALDEA', regex=True)
)

df['DIRECCION'] = df['DIRECCION'].str.replace(r'\s+', ' ', regex=True).str.strip()

df['DIRECCION'].replace({'': np.nan, 'NAN': np.nan}, inplace=True)

print(df['DIRECCION'].head(20))

0               KM.2 SALIDA A SAN JUAN CHAMELCO ZONA 8
1                         KM 209.5 ENTRADA A LA CIUDAD
2                            7A. AVENIDA 11-109 ZONA 6
3                                2A CALLE 11-10 ZONA 2
4                                  3A AVE 6-23 ZONA 11
5                                 5A. CALLE 1-9 ZONA 3
6                               11 AVENIDA 5-17 ZONA 4
7     DIAGONAL 08 8-05 ZONA 8, BARRIO CANTÓN LAS CASAS
8                                   12 AV. 2-12 ZONA 1
9                               5TA. CALLE 2-23 ZONA 4
10                              5TA. CALLE 2-23 ZONA 4
11                               3A. AVE. 6-23 ZONA 11
12                               3A. AVE. 6-23 ZONA 11
13                                   ALDEA LAS CONCHAS
14                             DIAGNONAL 1 1-26 ZONA 1
15                               5A. CALLE 1-06 ZONA 3
16                               7A. CALLE 1-13 ZONA 3
17                             4A. CALLE A 4-32 ZONA 8
18        

C:\Users\thiag\AppData\Local\Temp\ipykernel_9456\778544320.py:28: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['DIRECCION'].replace({'': np.nan, 'NAN': np.nan}, inplace=True)


# TELEFONO

In [ ]:
def limpiar_telefono_array(telefono):
    """
    Limpia y procesa los números de teléfono, convirtiendo TODO a formato array
    """
    if pd.isna(telefono):
        return np.nan
    
    # Convertir a string si es float o int
    if isinstance(telefono, (float, int)):
        telefono = str(int(telefono))
    else:
        telefono = str(telefono)
    
    # Remover espacios y caracteres no numéricos
    telefono_clean = re.sub(r'[^0-9]', '', telefono)
    
    # Si está vacío después de limpiar, retornar NaN
    if not telefono_clean:
        return np.nan
    
    # Si tiene exactamente 16 dígitos, dividir en dos números de 8
    if len(telefono_clean) == 16:
        num1 = telefono_clean[:8]
        num2 = telefono_clean[8:]
        
        # Verificar que ambos números tengan exactamente 8 dígitos
        if len(num1) == 8 and len(num2) == 8:
            return f"['{num1}', '{num2}']"
    
    # Para cualquier otro caso, crear un array con un solo elemento
    return f"['{telefono_clean}']"

# Aplicar la función de limpieza a todos los teléfonos
df['TELEFONO'] = df['TELEFONO'].apply(limpiar_telefono_array)
# Aca termina la limpieza de telefono :)

# Mostrar estadísticas
total_registros = len(df)
telefonos_validos = df['TELEFONO'].notna().sum()
telefonos_invalidos = df['TELEFONO'].isna().sum()

print(f"\nTotal de registros: {total_registros}")
print(f"Teléfonos válidos (arrays): {telefonos_validos}")
print(f"Teléfonos inválidos (NaN): {telefonos_invalidos}")

# Contar arrays simples vs dobles
telefono_series = df['TELEFONO'].dropna()
arrays_dobles = telefono_series.str.count("',").fillna(0) >= 1
arrays_simples = ~arrays_dobles

print(f"\nArrays con un teléfono: {arrays_simples.sum()}")
print(f"Arrays con dos teléfonos: {arrays_dobles.sum()}")

# Mostrar ejemplos
print("\n=== EJEMPLOS DE TELÉFONOS PROCESADOS ===")
print("Primeros 15 ejemplos:")
for i in range(min(15, len(df))):
    telefono = df['TELEFONO'].iloc[i]
    if pd.notna(telefono):
        print(f"Registro {i+1}: {telefono}")

# Mostrar algunos ejemplos de arrays dobles
print("\n=== EJEMPLOS DE ARRAYS DOBLES ===")
arrays_dobles_ejemplos = df[df['TELEFONO'].str.contains("',", na=False)]['TELEFONO'].head(5)
for i, tel in enumerate(arrays_dobles_ejemplos, 1):
    print(f"Ejemplo {i}: {tel}")



Total de registros: 6607
Teléfonos válidos (arrays): 6554
Teléfonos inválidos (NaN): 53

Arrays con un teléfono: 6522
Arrays con dos teléfonos: 32

=== EJEMPLOS DE TELÉFONOS PROCESADOS ===
Primeros 15 ejemplos:
Registro 1: ['77945104']
Registro 2: ['77367402']
Registro 3: ['78232301']
Registro 4: ['79514215']
Registro 5: ['79521468']
Registro 6: ['57101061']
Registro 7: ['79522555']
Registro 8: ['77930045']
Registro 9: ['79545566']
Registro 10: ['79514754']
Registro 11: ['79514754']
Registro 12: ['79521468']
Registro 13: ['79521468']
Registro 14: ['79522793']
Registro 15: ['51922050']

=== EJEMPLOS DE ARRAYS DOBLES ===
Ejemplo 1: ['79504027', '79504028']
Ejemplo 2: ['79540830', '79540909']
Ejemplo 3: ['79540830', '79540909']
Ejemplo 4: ['79649696', '78739432']
Ejemplo 5: ['78739432', '79649696']


# SUPERVISOR

In [6]:
def remove_accents_supervisor(text):
    """Elimina acentos y tildes de una cadena."""
    if pd.isna(text):
        return np.nan
    nkfd_form = unicodedata.normalize('NFKD', str(text))
    return ''.join(c for c in nkfd_form if not unicodedata.combining(c))

# 1. Convertir a string y normalizar espacios
df['SUPERVISOR'] = df['SUPERVISOR'].astype(str).str.strip()

# 2. Convertir a mayúsculas
df['SUPERVISOR'] = df['SUPERVISOR'].str.upper()

# 3. Quitar tildes y acentos
df['SUPERVISOR'] = df['SUPERVISOR'].apply(remove_accents_supervisor)

# 4. Reemplazar valores vacíos, 'NAN' y espacios en blanco con NaN
df['SUPERVISOR'] = df['SUPERVISOR'].replace({
    '': np.nan, 
    'NAN': np.nan,
    'NONE': np.nan,
    'NULL': np.nan
}, regex=False)

# También reemplazar strings que solo contengan espacios
df['SUPERVISOR'] = df['SUPERVISOR'].replace(r'^\s*$', np.nan, regex=True)

# Mostrar estadísticas
print(f"Total de registros: {len(df)}")
print(f"Supervisores válidos: {df['SUPERVISOR'].notna().sum()}")
print(f"Supervisores nulos (NaN): {df['SUPERVISOR'].isna().sum()}")

# Mostrar (primeros 20)
supervisores_unicos = df['SUPERVISOR'].dropna().unique()[:20]
for i, supervisor in enumerate(supervisores_unicos, 1):
    print(f"{i:2d}. {supervisor}")


Total de registros: 6607
Supervisores válidos: 6600
Supervisores nulos (NaN): 7
 1. PATRICIO NAJARRO ASENCIO
 2. NORA LILIANA FIGUEROA HERNANDEZ
 3. CESAR RIGOBERTO CUC CAAL
 4. MAXIMILIANO CHUB ICAL
 5. AQUILES GIOVANI BAC GUEVARA
 6. JORGE EDUARDO PAQUE LAZARO
 7. LEONARDO OXOM CHEN
 8. EMIILIO RUBEN CAAL GUALIB
 9. RUBEN DARIO CAAL CU
10. OLGA MARINA BUC
11. INGRID AMANDA CAMO TOBAR DE CASTRO
12. ENRIQUE HORACIO MOLINA MUNOZ
13. MARIA ELENA PALMA MOLINA
14. JOSE LUIS XI CUCUL
15. OSCAR GUILLERMO ANIBAL GARCIA HUMBLERS
16. ABNER DAVID CHIQUN TURCIOS
17. JORGE ERWIN CUCUL QUIB
18. DULLIER CLEMENTE BOL BAC
19. OSCAR RENE VENTURA ZAC
20. CECILIO DADAI OVANDO AZURDIA


# DIRECTOR

In [8]:
def remove_accents_director(text):
    """Elimina acentos y tildes de una cadena."""
    if pd.isna(text):
        return np.nan
    nkfd_form = unicodedata.normalize('NFKD', str(text))
    return ''.join(c for c in nkfd_form if not unicodedata.combining(c))

# 1. Convertir a string y normalizar espacios
df['DIRECTOR'] = df['DIRECTOR'].astype(str).str.strip()

# 2. Convertir a mayúsculas
df['DIRECTOR'] = df['DIRECTOR'].str.upper()

# 3. Quitar tildes y acentos
df['DIRECTOR'] = df['DIRECTOR'].apply(remove_accents_director)

# 4. Reemplazar valores vacíos, 'NAN' y espacios en blanco con NaN
df['DIRECTOR'] = df['DIRECTOR'].replace({
    '': np.nan, 
    'NAN': np.nan,
    'NONE': np.nan,
    'NULL': np.nan
}, regex=False)

# También reemplazar strings que solo contengan espacios
df['DIRECTOR'] = df['DIRECTOR'].replace(r'^\s*$', np.nan, regex=True)

# Mostrar estadísticas
print(f"Total de registros: {len(df)}")
print(f"Directores válidos: {df['DIRECTOR'].notna().sum()}")
print(f"Directores nulos (NaN): {df['DIRECTOR'].isna().sum()}")

# Mostrar valores únicos (primeros 20)
print(f"\nTotal de directores únicos: {df['DIRECTOR'].nunique()}")
print("\nPrimeros 20 directores únicos:")
directores_unicos = df['DIRECTOR'].dropna().unique()[:20]
for i, director in enumerate(directores_unicos, 1):
    print(f"{i:2d}. {director}")

# Mostrar ejemplos de transformación
print(f"\n=== EJEMPLOS DE DATOS PROCESADOS ===")
print("Primeros 10 registros después de limpieza:")
for i in range(min(10, len(df))):
    director = df['DIRECTOR'].iloc[i]
    print(f"Registro {i+1}: {director}")

Total de registros: 6607
Directores válidos: 6574
Directores nulos (NaN): 33

Total de directores únicos: 3825

Primeros 20 directores únicos:
 1. GUSTAVO ADOLFO SIERRA POP
 2. GILMA DOLORES GUAY PAZ DE LEAL
 3. VIRGINIA SOLANO SERRANO
 4. HECTOR ROLANDO CHUN POOU
 5. VICTOR HUGO DOMINGUEZ REYES
 6. MYNOR GUSTAVO IPINA ESPANA
 7. HECTOR ARMANDO TEYUL CHEN
 8. HELSON MARCO CHEN GUTIERREZ
 9. MARIA MAGDALENA BOL CU
10. JOSE CUPERTINO REYES GARCIA
11. DAVID WALDEMAR URRUTIA CHOC
12. SAMANTHA GABRIELA DE LA CRUZ MORALES
13. SILVIA BEATRIZ SOTO CHEN
14. JAYRON ARIEL COY CACAO
15. LESLY EUGENIA CHOCOOJ VIDAURRE
16. ZOYLA ARGENTINA MOLLINEDO ALVARADO
17. JUAN PASCUAL LUCAS ANTONIO
18. THELMA MARTINA POP TZIR
19. MARGARITA VILLALOBOS LIGORRIA
20. AXEL ABELARDO CHUB LEAL

=== EJEMPLOS DE DATOS PROCESADOS ===
Primeros 10 registros después de limpieza:
Registro 1: GUSTAVO ADOLFO SIERRA POP
Registro 2: GILMA DOLORES GUAY PAZ DE LEAL
Registro 3: VIRGINIA SOLANO SERRANO
Registro 4: HECTOR ROLANDO CH

# NIVEL

In [ ]:
# Análisis de la columna NIVEL
print("=== ANÁLISIS DE LA COLUMNA NIVEL ===")

# Verificar los valores únicos en la columna NIVEL
print("Valores únicos en NIVEL:")
valores_nivel = df['NIVEL'].value_counts(dropna=False)
print(valores_nivel)

print(f"\nTotal de registros: {len(df)}")
print(f"Registros con NIVEL válido: {df['NIVEL'].notna().sum()}")
print(f"Registros con NIVEL nulo (NaN): {df['NIVEL'].isna().sum()}")

# Mostrar algunos ejemplos de los valores
print(f"\nEjemplos de valores en NIVEL:")
ejemplos_nivel = df['NIVEL'].dropna().head(10)
for i, nivel in enumerate(ejemplos_nivel, 1):
    print(f"{i:2d}. {nivel}")

print("\n" + "="*80)
# Eliminar la columna NIVEL del DataFrame
print(" ELIMINANDO LA COLUMNA 'NIVEL'...")
df = df.drop('NIVEL', axis=1)

=== ANÁLISIS DE LA COLUMNA NIVEL ===
Valores únicos en NIVEL:
NIVEL
DIVERSIFICADO    6600
NaN                 7
Name: count, dtype: int64

Total de registros: 6607
Registros con NIVEL válido: 6600
Registros con NIVEL nulo (NaN): 7

Ejemplos de valores en NIVEL:
 1. DIVERSIFICADO
 2. DIVERSIFICADO
 3. DIVERSIFICADO
 4. DIVERSIFICADO
 5. DIVERSIFICADO
 6. DIVERSIFICADO
 7. DIVERSIFICADO
 8. DIVERSIFICADO
 9. DIVERSIFICADO
10. DIVERSIFICADO

RECOMENDACIÓN: ELIMINAR LA COLUMNA NIVEL
 ELIMINANDO LA COLUMNA 'NIVEL'...



## JUSTIFICACIÓN PARA ELIMINAR LA COLUMNA 'NIVEL':

1. FALTA DE VARIABILIDAD: Una columna que contiene un solo valor constante no aporta información discriminatoria para el análisis, ya que no permite segmentar o diferenciar los datos.

2. REDUNDANCIA: Los valores NaN probablemente también corresponden a instituciones  de nivel diversificado, pero con información faltante, no a niveles diferentes.

3. OPTIMIZACIÓN: Eliminar columnas constantes reduce la dimensionalidad del dataset sin pérdida de información relevante, mejorando la eficiencia del procesamiento.

4. CLARIDAD EN EL ANÁLISIS: Al eliminar esta columna, queda claro que todo el análisis se enfoca exclusivamente en instituciones de nivel diversificado.


# SECTOR

In [27]:
df["SECTOR"].unique()

array(['PRIVADO', 'OFICIAL', 'MUNICIPAL', 'COOPERATIVA', nan],
      dtype=object)

In [31]:
# Mostrar valores únicos antes de la limpieza
print("Valores únicos ANTES de la limpieza:")
print(df['SECTOR'].unique())
print(f"Valores únicos count: {df['SECTOR'].nunique()}")

# 1. Convertir a string y normalizar espacios
df['SECTOR'] = df['SECTOR'].astype(str).str.strip()

# 2. Convertir todo a mayúsculas
df['SECTOR'] = df['SECTOR'].str.upper()

# 3. Reemplazar valores vacíos, 'NAN' y espacios en blanco con NaN
df['SECTOR'] = df['SECTOR'].replace({
    '': np.nan, 
    'NAN': np.nan,
    'NONE': np.nan,
    'NULL': np.nan,
    "nan": np.nan
}, regex=False)

# También reemplazar strings que solo contengan espacios
df['SECTOR'] = df['SECTOR'].replace(r'^\s*$', np.nan, regex=True)

# Mostrar valores únicos después de la limpieza
print("\nValores únicos DESPUÉS de la limpieza:")
print(df['SECTOR'].unique())
print(f"Valores únicos count: {df['SECTOR'].nunique()}")

# Mostrar estadísticas
print(f"\nTotal de registros: {len(df)}")
print(f"Sectores válidos: {df['SECTOR'].notna().sum()}")
print(f"Sectores nulos (NaN): {df['SECTOR'].isna().sum()}")

# Mostrar conteo por sector
print(f"\nDistribución por SECTOR:")
sector_counts = df['SECTOR'].value_counts(dropna=False)
for sector, count in sector_counts.items():
    print(f"  {sector}: {count} registros")

print("\nLimpieza de SECTOR completada")

Valores únicos ANTES de la limpieza:
['PRIVADO' 'OFICIAL' 'MUNICIPAL' 'COOPERATIVA' nan]
Valores únicos count: 4

Valores únicos DESPUÉS de la limpieza:
['PRIVADO' 'OFICIAL' 'MUNICIPAL' 'COOPERATIVA' nan]
Valores únicos count: 4

Total de registros: 6607
Sectores válidos: 6600
Sectores nulos (NaN): 7

Distribución por SECTOR:
  PRIVADO: 5418 registros
  OFICIAL: 874 registros
  COOPERATIVA: 213 registros
  MUNICIPAL: 95 registros
  nan: 7 registros

Limpieza de SECTOR completada


# AREA

# STATUS

### No es necesario transformar nada, solo realizar relleno de nan

In [160]:
df['STATUS'] = df['STATUS'].replace(r'^\s*$', np.nan, regex=True)

In [161]:
df['STATUS']

0       ABIERTA
1       ABIERTA
2       ABIERTA
3       ABIERTA
4       ABIERTA
         ...   
6602    ABIERTA
6603    ABIERTA
6604    ABIERTA
6605    ABIERTA
6606    ABIERTA
Name: STATUS, Length: 6607, dtype: object

In [162]:
df['STATUS'].isna().sum()

np.int64(7)

In [163]:
df['STATUS'].value_counts()

STATUS
ABIERTA    6600
Name: count, dtype: int64

# MODALIDAD

- Verificar consistencia en mayúsculas (convertir todo a mayúsculas). 
- Revisar posibles variantes (e.g., “Monolingüe” con tilde) y homogeneizar (reemplazar tildes: “MONOLINGUE”). 
- Confirmar que no existan valores nulos o espacios en blanco (pasarlos a NULL). 

In [164]:
# Eliminar espacios y convierte a mayúsculas
df['MODALIDAD'] = df['MODALIDAD'].astype(str).str.strip().str.upper()

In [165]:
# Quita tildes (acentos) para homogeneizar
df['MODALIDAD'] = df['MODALIDAD'].apply(
    lambda x: unicodedata
        .normalize('NFKD', x)
        .encode('ASCII', 'ignore')
        .decode('ASCII')
)

In [166]:
# Detecta cadenas vacías o sólo espacios, pásalas a NaN
df['MODALIDAD'] = df['MODALIDAD'].replace(r'^\s*$', np.nan, regex=True)

In [167]:
print(df['MODALIDAD'].unique())

['MONOLINGUE' 'BILINGUE' 'NAN']


In [168]:
df['MODALIDAD'].isna().sum()

np.int64(0)

In [169]:
df['MODALIDAD'].value_counts()

MODALIDAD
MONOLINGUE    6391
BILINGUE       209
NAN              7
Name: count, dtype: int64

# JORNADA

- Convertir todo a mayúsculas y recortar espacios (strip). 
- Normalizar nombres (e.g., unificar “SIN JORNADA” y “SINJORNADA”). 
- Validar categorías permitidas (hacer un “merge” con lista de categorías oficiales y asignar “OTRA” a valores inválidos o not-matched). 

In [170]:
# strip + uppercase
df['JORNADA'] = df['JORNADA'].astype(str).str.strip().str.upper()

In [171]:
print(df['JORNADA'].unique())

['MATUTINA' 'VESPERTINA' 'DOBLE' 'NOCTURNA' 'SIN JORNADA' 'NAN'
 'INTERMEDIA']


In [172]:
# Normalizar variantes conocidas
df['JORNADA'] = df['JORNADA'].replace({
    'SINJORNADA':    'SIN JORNADA',
    'SIN JORNADA':  'SINJORNADA',  
})

In [173]:
print([repr(v) for v in df['JORNADA'].unique()])

["'MATUTINA'", "'VESPERTINA'", "'DOBLE'", "'NOCTURNA'", "'SINJORNADA'", "'NAN'", "'INTERMEDIA'"]


In [174]:
df['JORNADA'] = (
    df['JORNADA']
    .replace('NAN', np.nan)                   
    .replace(r'^\s*$', np.nan, regex=True)    
)

In [175]:
df['JORNADA'].isna().sum()

np.int64(7)

In [176]:
df['JORNADA'].value_counts()

JORNADA
DOBLE         1957
VESPERTINA    1836
MATUTINA      1687
SINJORNADA    1001
NOCTURNA       106
INTERMEDIA      13
Name: count, dtype: int64

# PLAN

- Convertir a mayúsculas y eliminar tildes (ejemplo: DÍA → DIA). 
- Estandarizar nomenclatura de semipresencial: extraer descriptor entre paréntesis a nueva variable o bien dejar solo “SEMIPRESENCIAL” y crear columna secundaria para “PERIODICIDAD” (FIN_DE_SEMANA, UN_DIA_SEMANA, DOS_DIAS_SEMANA). 
- Eliminar espacios redundantes y caracteres no alfanuméricos innecesarios. 

In [177]:
print(df['PLAN'].unique())

['DIARIO(REGULAR)' 'FIN DE SEMANA' 'A DISTANCIA' 'SEMIPRESENCIAL'
 'SEMIPRESENCIAL (FIN DE SEMANA)' 'SEMIPRESENCIAL (UN DÍA A LA SEMANA)'
 'VIRTUAL A DISTANCIA' nan 'SEMIPRESENCIAL (DOS DÍAS A LA SEMANA)'
 'SABATINO' 'INTERCALADO' 'DOMINICAL' 'MIXTO']


In [178]:
# Función para quitar tildes
def remove_accents(s: str) -> str:
    return unicodedata.normalize('NFKD', s) \
                      .encode('ASCII', 'ignore') \
                      .decode('ASCII')

In [179]:
# Limpieza de la columna PLAN en sitio
df['PLAN'] = (
    df['PLAN']
      .astype(str)                                    
      .where(df['PLAN'].notna(), np.nan)              
      .str.strip()                                     
      .map(lambda x: remove_accents(x) if pd.notna(x) else x)  
      .str.upper()                                     
      .str.replace(r'\(.*?\)', '', regex=True)         
      .str.replace(r'[^A-Z0-9 ]+', '', regex=True)     
      .str.replace(r'\s+', ' ', regex=True)            
      .str.strip()                                     
      .replace({'': np.nan, 'NAN': np.nan})            
)


In [180]:
print(df['PLAN'].unique())

['DIARIO' 'FIN DE SEMANA' 'A DISTANCIA' 'SEMIPRESENCIAL'
 'VIRTUAL A DISTANCIA' nan 'SABATINO' 'INTERCALADO' 'DOMINICAL' 'MIXTO']


In [181]:
df['PLAN'].value_counts()

PLAN
DIARIO                 4052
FIN DE SEMANA          1310
SEMIPRESENCIAL         1060
A DISTANCIA             116
VIRTUAL A DISTANCIA      52
SABATINO                  5
INTERCALADO               2
DOMINICAL                 2
MIXTO                     1
Name: count, dtype: int64

# DEPARTAMENTAL

- Convertir a mayúsculas y recortar espacios. 
- Reemplazar guiones o caracteres especiales por espacios. 
- Comparar con lista oficial de regiones y corregir discrepancias (p.ej., “GUATEMALA NORTE”). 

In [182]:
df['DEPARTAMENTAL'] = df['DEPARTAMENTAL'].astype(str).str.strip().str.upper()

In [183]:
# Reemplazar guiones o underscores por espacios, y quitar caracteres especiales
df['DEPARTAMENTAL'] = (
    df['DEPARTAMENTAL']
      .str.replace(r'[-_]+', ' ', regex=True)
      .str.replace(r'[^A-Z0-9 ]+', ' ', regex=True)
      .str.replace(r'\s+', ' ', regex=True)
      .str.strip()
)

In [184]:
#Lista oficial de departamentos
official = [
    'ALTA VERAPAZ','BAJA VERAPAZ','CHIMALTENANGO','CHIQUIMULA','EL PROGRESO',
    'ESCUINTLA','GUATEMALA','HUEHUETENANGO','IZABAL','JALAPA','JUTIAPA',
    'PETEN','QUICHE','RETALHULEU','SACATEPEQUEZ','SANTA ROSA','SOLOLÁ',
    'SUCHITEPEQUEZ','TOTONICAPÁN','QUETZALTENANGO','SAN MARCOS'
]

In [185]:
mapping = {
    'GUATEMALA NORTE': 'GUATEMALA',
    'GUATEMALA SUR':   'GUATEMALA',
    'GUATEMALA ESTE':   'GUATEMALA',
    'GUATEMALA OESTE':   'GUATEMALA',
}

df['DEPARTAMENTAL'] = df['DEPARTAMENTAL'].replace(mapping)

In [186]:
df['DEPARTAMENTAL'] = df['DEPARTAMENTAL'].where(df['DEPARTAMENTAL'].isin(official), np.nan)

In [187]:
df['DEPARTAMENTAL'].value_counts()

DEPARTAMENTAL
GUATEMALA         1073
SAN MARCOS         432
ESCUINTLA          393
QUETZALTENANGO     365
CHIMALTENANGO      304
JUTIAPA            296
HUEHUETENANGO      295
ALTA VERAPAZ       294
IZABAL             273
RETALHULEU         272
CHIQUIMULA         136
SANTA ROSA         133
JALAPA             121
EL PROGRESO         97
BAJA VERAPAZ        94
Name: count, dtype: int64

# Eliminacion de filas con 10 o mas datos null

In [188]:
df

,CODIGO,DISTRITO,DEPARTAMENTO,MUNICIPIO,ESTABLECIMIENTO,DIRECCION,TELEFONO,SUPERVISOR,DIRECTOR,NIVEL,SECTOR,AREA,STATUS,MODALIDAD,JORNADA,PLAN,DEPARTAMENTAL
0,16-01-0138-46,16-031,ALTA VERAPAZ,COBAN,COLEGIO COBAN,KM.2 SALIDA A SAN JUAN CHAMELCO ZONA 8,77945104,PATRICIO NAJARRO ASENCIO,GUSTAVO ADOLFO SIERRA POP,DIVERSIFICADO,PRIVADO,URBANA,ABIERTA,MONOLINGUE,MATUTINA,DIARIO,ALTA VERAPAZ
1,16-01-0139-46,16-031,ALTA VERAPAZ,COBAN,COLEGIO PARTICULAR MIXTO VERAPAZ,KM 209.5 ENTRADA A LA CIUDAD,77367402,PATRICIO NAJARRO ASENCIO,GILMA DOLORES GUAY PAZ DE LEAL,DIVERSIFICADO,PRIVADO,URBANA,ABIERTA,MONOLINGUE,MATUTINA,DIARIO,ALTA VERAPAZ
2,16-01-0140-46,16-031,ALTA VERAPAZ,COBAN,COLEGIO LA INMACULADA,7A. AVENIDA 11-109 ZONA 6,78232301,PATRICIO NAJARRO ASENCIO,VIRGINIA SOLANO SERRANO,DIVERSIFICADO,PRIVADO,URBANA,ABIERTA,MONOLINGUE,MATUTINA,DIARIO,ALTA VERAPAZ
3,16-01-0141-46,16-005,ALTA VERAPAZ,COBAN,ESCUELA NACIONAL DE CIENCIAS COMERCIALES,2A CALLE 11-10 ZONA 2,79514215,NORA LILIANA FIGUEROA HERNÁNDEZ,HÉCTOR ROLANDO CHUN POOU,DIVERSIFICADO,OFICIAL,URBANA,ABIERTA,MONOLINGUE,MATUTINA,DIARIO,ALTA VERAPAZ
4,16-01-0142-46,16-005,ALTA VERAPAZ,COBAN,INSTITUTO NORMAL MIXTO DEL NORTE EMILIO ROSALE...,3A AVE 6-23 ZONA 11,79521468,NORA LILIANA FIGUEROA HERNÁNDEZ,VICTOR HUGO DOMÍNGUEZ REYES,DIVERSIFICADO,OFICIAL,URBANA,ABIERTA,BILINGUE,VESPERTINA,DIARIO,ALTA VERAPAZ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6602,19-09-0040-46,19-021,ZACAPA,LA UNION,LICEO PARTICULAR MIXTO JIREH,BARRIO NUEVO,79418369,ASBEL IVÁN SÚCHITE ARROYO,ANA MARÍA CUELLAR GUERRA,DIVERSIFICADO,PRIVADO,URBANA,ABIERTA,MONOLINGUE,MATUTINA,DIARIO,NaN
6603,19-09-0048-46,19-021,ZACAPA,LA UNION,LICEO PARTICULAR MIXTO JIREH,BARRIO NUEVO,79418369,ASBEL IVÁN SÚCHITE ARROYO,ANA MARÍA CUELLAR GUERRA,DIVERSIFICADO,PRIVADO,URBANA,ABIERTA,MONOLINGUE,SINJORNADA,SEMIPRESENCIAL,NaN
6604,19-10-0013-46,19-015,ZACAPA,HUITE,INSTITUTO DIVERSIFICADO,BARRIO BUENOS AIRES,48579171,SILDY MARIELA PEREZ FRANCO,WUENDY JHOJANA SIERRA PAZ,DIVERSIFICADO,OFICIAL,URBANA,ABIERTA,MONOLINGUE,NOCTURNA,DIARIO,NaN
6605,19-10-1009-46,19-015,ZACAPA,HUITE,INSTITUTO DIVERSIFICADO POR COOPERATIVA,BARRIO EL CAMPO,55958103,SILDY MARIELA PEREZ FRANCO,ROBIDIO PORTILLO SALGUERO,DIVERSIFICADO,COOPERATIVA,URBANA,ABIERTA,MONOLINGUE,VESPERTINA,DIARIO,NaN


In [189]:
#Cuenta los valores faltantes por fila
missing_per_row = df.isna().sum(axis=1)

In [190]:
#Crea una máscara para las filas con 10 o más valores faltantes
mask = missing_per_row >= 10

In [191]:
n_rows = mask.sum()
print(f'{n_rows} filas tienen 10 o más valores faltantes.')

7 filas tienen 10 o más valores faltantes.


In [192]:
df_clean = df.loc[~mask].reset_index(drop=True)

In [193]:
df_clean

,CODIGO,DISTRITO,DEPARTAMENTO,MUNICIPIO,ESTABLECIMIENTO,DIRECCION,TELEFONO,SUPERVISOR,DIRECTOR,NIVEL,SECTOR,AREA,STATUS,MODALIDAD,JORNADA,PLAN,DEPARTAMENTAL
0,16-01-0138-46,16-031,ALTA VERAPAZ,COBAN,COLEGIO COBAN,KM.2 SALIDA A SAN JUAN CHAMELCO ZONA 8,77945104,PATRICIO NAJARRO ASENCIO,GUSTAVO ADOLFO SIERRA POP,DIVERSIFICADO,PRIVADO,URBANA,ABIERTA,MONOLINGUE,MATUTINA,DIARIO,ALTA VERAPAZ
1,16-01-0139-46,16-031,ALTA VERAPAZ,COBAN,COLEGIO PARTICULAR MIXTO VERAPAZ,KM 209.5 ENTRADA A LA CIUDAD,77367402,PATRICIO NAJARRO ASENCIO,GILMA DOLORES GUAY PAZ DE LEAL,DIVERSIFICADO,PRIVADO,URBANA,ABIERTA,MONOLINGUE,MATUTINA,DIARIO,ALTA VERAPAZ
2,16-01-0140-46,16-031,ALTA VERAPAZ,COBAN,COLEGIO LA INMACULADA,7A. AVENIDA 11-109 ZONA 6,78232301,PATRICIO NAJARRO ASENCIO,VIRGINIA SOLANO SERRANO,DIVERSIFICADO,PRIVADO,URBANA,ABIERTA,MONOLINGUE,MATUTINA,DIARIO,ALTA VERAPAZ
3,16-01-0141-46,16-005,ALTA VERAPAZ,COBAN,ESCUELA NACIONAL DE CIENCIAS COMERCIALES,2A CALLE 11-10 ZONA 2,79514215,NORA LILIANA FIGUEROA HERNÁNDEZ,HÉCTOR ROLANDO CHUN POOU,DIVERSIFICADO,OFICIAL,URBANA,ABIERTA,MONOLINGUE,MATUTINA,DIARIO,ALTA VERAPAZ
4,16-01-0142-46,16-005,ALTA VERAPAZ,COBAN,INSTITUTO NORMAL MIXTO DEL NORTE EMILIO ROSALE...,3A AVE 6-23 ZONA 11,79521468,NORA LILIANA FIGUEROA HERNÁNDEZ,VICTOR HUGO DOMÍNGUEZ REYES,DIVERSIFICADO,OFICIAL,URBANA,ABIERTA,BILINGUE,VESPERTINA,DIARIO,ALTA VERAPAZ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6595,19-09-0040-46,19-021,ZACAPA,LA UNION,LICEO PARTICULAR MIXTO JIREH,BARRIO NUEVO,79418369,ASBEL IVÁN SÚCHITE ARROYO,ANA MARÍA CUELLAR GUERRA,DIVERSIFICADO,PRIVADO,URBANA,ABIERTA,MONOLINGUE,MATUTINA,DIARIO,NaN
6596,19-09-0048-46,19-021,ZACAPA,LA UNION,LICEO PARTICULAR MIXTO JIREH,BARRIO NUEVO,79418369,ASBEL IVÁN SÚCHITE ARROYO,ANA MARÍA CUELLAR GUERRA,DIVERSIFICADO,PRIVADO,URBANA,ABIERTA,MONOLINGUE,SINJORNADA,SEMIPRESENCIAL,NaN
6597,19-10-0013-46,19-015,ZACAPA,HUITE,INSTITUTO DIVERSIFICADO,BARRIO BUENOS AIRES,48579171,SILDY MARIELA PEREZ FRANCO,WUENDY JHOJANA SIERRA PAZ,DIVERSIFICADO,OFICIAL,URBANA,ABIERTA,MONOLINGUE,NOCTURNA,DIARIO,NaN
6598,19-10-1009-46,19-015,ZACAPA,HUITE,INSTITUTO DIVERSIFICADO POR COOPERATIVA,BARRIO EL CAMPO,55958103,SILDY MARIELA PEREZ FRANCO,ROBIDIO PORTILLO SALGUERO,DIVERSIFICADO,COOPERATIVA,URBANA,ABIERTA,MONOLINGUE,VESPERTINA,DIARIO,NaN


In [194]:
print(f"Filas originales: {len(df)}")
print(f"Filas tras eliminar ≥10 faltantes: {len(df_clean)}")

Filas originales: 6607
Filas tras eliminar ≥10 faltantes: 6600
